# sensor_correlations.ipynb
Functions to examine rolling correlations between device sensor outputs.
Author: – Jon Clucas, 2017 jon.clucas@childmind.org
© 2017, Child Mind Institute, Apache v2.0 License

setup:

In [ ]:
%matplotlib inline
from annotate_range import annotation_line
from astropy.stats import median_absolute_deviation as mad
from chart_data import write_csv
from config import organized_dir, placement_dir
from datetime import datetime, timedelta
from matplotlib.dates import DateFormatter
from plot_normalized_vector_lengths import baseshift_and_renormalize
import json, numpy as np, os, pandas as pd, matplotlib.pyplot as plt
with open(os.path.join('./line_charts/device_colors.json')) as fp:
    color_key = json.load(fp)
pd.set_option('mode.use_inf_as_null', True)

define functions:

In [ ]:
def bland_altman_plot(data1, data2, *args, **kwargs):
    data1     = np.asarray(data1)
    data2     = np.asarray(data2)
    mean      = np.mean([data1, data2], axis=0)
    diff      = data1 - data2                   # Difference between data1 and data2
    md        = np.mean(diff)                   # Mean of the difference
    sd        = np.std(diff, axis=0)            # Standard deviation of the difference

    plt.scatter(mean, diff, *args, **kwargs)
    plt.axhline(md,           color='gray', linestyle='--')
    plt.axhline(md + 1.96*sd, color='gray', linestyle='--')
    plt.axhline(md - 1.96*sd, color='gray', linestyle='--')

def linechart(df, plot_label, plot_person, line=True):
    """
    Function to build a linechart and export a PNG and an SVG of the image.
    
    Parameters
    ----------
    df : pandas dataframe
        dataframe to plot
        
    plot_label : string
        plot title
        
    plot_person : string
        wearer (for annotations)
        
    Returns
    -------
    None
    
    Outputs
    -------
    inline plot
    """
    start = min(df.index.values)
    stop = max(df.index.values)
    w_log = pd.read_csv(os.path.join(placement_dir, 'wearable_log.csv'),
            parse_dates={'start':['start date', 'start time'], 'stop':[
            'end date', 'end time']})
    w_log.dropna(inplace=True)
    w_log['start'] = pd.to_datetime(w_log.start, errors="coerce")
    w_log['stop'] = pd.to_datetime(w_log.stop, errors="coerce")
    print("Plotting...")
    print(plot_label)
    svg_out = os.path.join(organized_dir, 'accelerometer', "_".join([
              'normalized_vector_length', '.'.join(['_'.join(plot_label.split(
              ' ')), 'svg'])]))
    png_out = ''.join([svg_out.strip('svg'), 'png'])
    fig = plt.figure(figsize=(10, 8), dpi=75)
    plt.rcParams['agg.path.chunksize'] = 10000
    ax = fig.add_subplot(111)
    ax.set_ylabel('unit cube normalized vector length')
    annotations_a = {}
    annotations_b = {}
    annotation_y = 0.04
    plot_log = w_log.loc[(w_log['wearer'] == plot_person) & (w_log['start'] >= 
                         start) & (w_log['stop'] <= stop)].copy()
    for row in plot_log.itertuples():
        if row[4] not in annotations_a:
            annotations_a[row[4]] = row[1] if row[1] >= start else start
            annotations_b[row[4]] = row[2] if row[2] <= stop else stop
    esses = []
    mad_values = []
    for device in list(df.columns):
        if device.startswith('normalized'):
            d2 = device[25:]
        else:
            d2 = device
        plot_line = df[[device]].dropna()
        print(mad(plot_line))
        mad_values.append(mad(plot_line))
        esses.append(pd.Series(plot_line.iloc[:,0], name=d2, index=
                     plot_line.index))
        if "GENEActiv" in device:
            label = "GENEActiv"
        elif device == "Actigraph":
            label = "ActiGraph"
        else:
            label = d2
        """
        if device == "Wavelet":
            ax.plot_date(x=plot_line.index, y=plot_line, color=color_key[
                         device], alpha=0.5, label=label, marker="o",
                         linestyle="None")
        else:
        """
        if line:
            ax.plot_date(x=plot_line.index, y=plot_line, color=color_key[
                             d2], alpha=0.5, label=label, marker="", linestyle=
                             "solid")
        else:
            ax.plot_date(x=plot_line.index, y=plot_line, color=color_key[
                             d2], alpha=0.5, label=label, marker="o", linestyle=
                             "None")
        ax.legend(loc='best', fancybox=True, framealpha=0.5)
    for annotation in annotations_a:
        try:
            annotation_line(ax, annotations_a[annotation], annotations_b[
                            annotation], annotation, annotation_y)
            annotation_y += 0.08
        except:
            print(annotation)
    ylim = max(mad_values)
    if(ylim == 0):
        ax.set_ylim([0, 1])
    else:
        ax.set_ylim([0, 5 * ylim])
    ax.xaxis.set_major_formatter(DateFormatter('%H:%M:%S'))
    plt.suptitle(plot_label)
    plt.xticks(rotation=65)
    plt.show()
    
def df_two_devices(devices, sensor, start, stop):
    """
    Function to calculate rolling correlations between two sensor data streams.
    
    Parameters
    ----------
    devices : list of strings (len 2)
        each string is the name of one of the two devices to compare
        
    sensor : string
        the sensor to compare
        
    start : datetime
        beginning of time to compare
        
    stop : datetime
        end of time to compare
        
    Returns
    -------
    df : pandas dataframe
        merged dataframe with a column per device
    """
    if sensor == 'accelerometer':
        suffix = '_normalized_unit.csv'
    else:
        suffix = '.csv'
    s0 = pd.read_csv(os.path.join(organized_dir, sensor, ''.join([devices[0],
         suffix])), usecols=['Timestamp', 'normalized_vector_length'],
         parse_dates=['Timestamp'], infer_datetime_format=True)
    s0 = s0.loc[(s0['Timestamp'] >= start) & (s0['Timestamp'] <= stop)].copy()
    s1 = pd.read_csv(os.path.join(organized_dir, sensor, ''.join([devices[1],
         suffix])), usecols=['Timestamp', 'normalized_vector_length'],
         parse_dates=['Timestamp'], infer_datetime_format=True)
    s1 = s1.loc[(s1['Timestamp'] >= start) & (s1['Timestamp'] <= stop)].copy()
    # m0, m1 = only_matching_ts_points(s0, s1, 'second')
    s0 = baseshift_and_renormalize(s0)
    s1 = baseshift_and_renormalize(s1)
    s1[['Timestamp']] = s1.Timestamp.apply(lambda x: x - timedelta(microseconds
                        =1000))
    s0.set_index('Timestamp', inplace=True)
    s1.set_index('Timestamp', inplace=True)
    df = s0.merge(s1, left_index=True, right_index=True, suffixes=(''.join([
         '_', devices[0]]), ''.join(['_', devices[1]])))
    return(df)

In [ ]:
df = df_two_devices(['GENEActiv_pink', 'Actigraph'], 'accelerometer',  datetime(2017, 4, 6, 15, 45), datetime(2017, 4,
     7, 14, 8))

In [ ]:
linechart(df, "ActiGraph vs GENEActiv, 1 day", "Arno")

In [ ]:
start = datetime(2017, 4, 6, 15, 45)
# stop = datetime(2017, 4, 7, 14, 8)
stop = datetime(2017, 4, 6, 16, 45)
while start < stop:
    new_start = start + timedelta(seconds=3)
    plot_df = df.loc[(df.index >= start) & (df.index <= new_start)].copy()
    label = '–'.join([start.strftime('%H:%M:%S'), new_start.strftime('%H:%M:%S')])
    linechart(plot_df, label, "Arno", False)
    start = new_start

In [ ]:
bland_altman_plot(df.normalized_vector_length_GENEActiv_pink, df.normalized_vector_length_Actigraph)

In [ ]:
bland_altman_plot(df.normalized_vector_length_GENEActiv_pink, df.normalized_vector_length_Actigraph, ylim=0.2)

In [ ]:
pdcorr = df.corr(method='pearson')
print(pdcorr)

In [ ]:
start = datetime(2017, 4, 7, 7, 16)
stop = start + timedelta(seconds=30)
sliced_df = df.loc[(df.index >= start) & (df.index <= stop)].copy()
print(sliced_df.corr(method='pearson'))

In [ ]:
linechart(df, "ActiGraph vs GENEActiv", "Arno")

In [ ]:
rcorr = df.rolling(window=180, center=True).corr()
print(rcorr.dropna().mean())

In [ ]:
no_outliers = df.copy()
print(len(no_outliers))
for column in list(no_outliers.columns):
    truths = (np.abs(no_outliers[[column]] - no_outliers[[column]].mean()) <= (3 * no_outliers[[column]].std()))
    no_outliers[[column]] = no_outliers[[column]][truths]
no_outliers.dropna(how='any', inplace=True)
print(len(no_outliers))

In [ ]:
pdcorr = no_outliers.corr(method='pearson')
print(pdcorr)